<a href="https://colab.research.google.com/github/Thejus-1503/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Import the data sets
movie_df = pd.read_csv('/content/drive/MyDrive/movies.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/ratings.csv')

In [3]:
# Preview the movie dataset
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# Preview the rating dataset
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,5.0,847117005
1,1,2,3.0,847642142
2,1,10,3.0,847641896
3,1,32,4.0,847642008
4,1,34,4.0,847641956


In [5]:
# Let's combine both datasets to df
df = pd.merge(rating_df,movie_df,on='movieId', how='left')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,5.0,847117005,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,3.0,847642142,Jumanji (1995),Adventure|Children|Fantasy
2,1,10,3.0,847641896,GoldenEye (1995),Action|Adventure|Thriller
3,1,32,4.0,847642008,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
4,1,34,4.0,847641956,Babe (1995),Children|Drama


In [6]:
# Let's drop timestamp since it doesnot provide much for the recommendation
df.drop('timestamp',axis=1,inplace=True)
df.head()

,userId,movieId,rating,title,genres
0,1,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,10,3.0,GoldenEye (1995),Action|Adventure|Thriller
3,1,32,4.0,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
4,1,34,4.0,Babe (1995),Children|Drama


In [7]:
# check for null
df.isnull().sum()

,0
userId,0
movieId,0
rating,0
title,0
genres,0


In [8]:
# Check for duplicates
print("Duplicate rows:", df.duplicated().sum())

Duplicate rows: 0


In [9]:
# Create the User-Movie Matrix and fill with zeros
# Group by userId and title and take the mean of the ratings to handle duplicates
movie_user_rating_pivot = df.groupby(['userId', 'title'])['rating'].mean().unstack().fillna(0)
movie_user_rating_pivot.head(10)

title,"""Great Performances"" Cats (1998)",'71 (2014),'Round Midnight (1986),'Til There Was You (1997),'night Mother (1986),(500) Days of Summer (2009),(Untitled) (2009),*batteries not included (1987),...And Justice for All (1979),1-900 (06) (1994),...,Zoom (2006),Zorro the Gay Blade (1981),Zulu (1964),[REC] (2007),[REC]�_ (2009),burbs The (1989),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),��Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,4.0,3.5,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X = movie_user_rating_pivot.values.T
X.shape

(8913, 718)

In [11]:
# Matrix Factorization Using NumPy (Gradient Descent)
# Convert pivot table to NumPy array
R = movie_user_rating_pivot.values
num_users, num_items = R.shape

def matrix_factorization(R, K=10, steps=100, alpha=0.002, beta=0.02):
    M, N = R.shape
    P = np.random.rand(M, K)
    Q = np.random.rand(N, K)
    Q = Q.T

    for step in range(steps):
        for i in range(M):
            for j in range(N):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] += alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] += alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        # Optional: Print error every 10 steps
        if step % 10 == 0:
            error = np.sum((R[R > 0] - np.dot(P, Q) [R > 0])**2)
            print(f"Iteration {step}, Error: {error:.2f}")
    return P, Q.T

In [12]:
# Train the Matrix Factorization Model
P, Q = matrix_factorization(R, K=15, steps=100)
predicted_ratings = np.dot(P, Q.T)

Iteration 0, Error: 98749.64
Iteration 10, Error: 67968.33
Iteration 20, Error: 60274.10
Iteration 30, Error: 53555.36
Iteration 40, Error: 47662.40
Iteration 50, Error: 42916.71
Iteration 60, Error: 39202.85
Iteration 70, Error: 36295.67
Iteration 80, Error: 33999.99
Iteration 90, Error: 32164.87


In [13]:
# Build the Predicted Ratings DataFrame
predicted_df = pd.DataFrame(predicted_ratings,
                            index=movie_user_rating_pivot.index,
                            columns=movie_user_rating_pivot.columns)
predicted_df.head()

title,"""Great Performances"" Cats (1998)",'71 (2014),'Round Midnight (1986),'Til There Was You (1997),'night Mother (1986),(500) Days of Summer (2009),(Untitled) (2009),*batteries not included (1987),...And Justice for All (1979),1-900 (06) (1994),...,Zoom (2006),Zorro the Gay Blade (1981),Zulu (1964),[REC] (2007),[REC]�_ (2009),burbs The (1989),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),��Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.790125,4.550517,3.354614,4.037877,3.966013,4.379734,2.714191,3.718719,3.691164,4.411331,...,3.121628,3.498203,4.386872,4.409307,3.663297,3.019126,3.763191,2.624204,2.786769,3.387827
2,3.897650,4.270591,3.814822,3.156316,3.175409,3.679142,2.353130,3.261210,3.238462,4.711982,...,2.757540,4.396679,3.947752,3.723516,3.427435,2.800646,3.451324,2.743482,2.300476,2.649576
3,3.747934,3.901037,3.867816,3.907944,3.465810,4.623734,3.108128,3.313106,3.913856,4.598670,...,3.418706,3.771229,4.305645,5.223042,2.922635,2.601878,4.129918,2.676946,3.244155,3.283166
4,3.455740,4.346500,2.990638,4.497423,4.012308,3.787888,2.899113,3.077226,3.249273,4.406661,...,2.722309,3.853003,3.756633,4.477714,4.139395,2.855912,3.748377,2.784694,2.714379,3.435270
5,3.316336,4.884438,2.807351,4.723013,3.473435,3.461137,2.588448,3.691065,4.141569,4.519369,...,2.707915,2.717407,4.490116,4.362789,4.126171,2.954615,3.031096,2.185921,2.427243,3.832900


In [19]:
# Recommend Top 5 Movies for a User
def recommend_movies(user_id, actual_df, predicted_df, n=5):
    user_row = predicted_df.loc[user_id]
    already_rated = actual_df.loc[user_id]
    unrated = already_rated[already_rated == 0].index
    recommendations = user_row[unrated].sort_values(ascending=False).head(n)
    return recommendations

# Example: Recommend 5 movies to user 7
top_movies = recommend_movies(user_id=7, actual_df=movie_user_rating_pivot, predicted_df=predicted_df, n=5)
print("Top 5 Movie Recommendations for User 7:\n")
print(top_movies)

Top 5 Movie Recommendations for User 7:

title
Prophet  A (Un Proph̬te) (2009)       5.810392
Tenant  The (Locataire  Le) (1976)    5.703838
Kenny (2006)                          5.531592
Dead  The (1987)                      5.495840
Starred Up (2013)                     5.479374
Name: 7, dtype: float64


In [15]:
# Recommend Top 5 Movies based on a perticular geners

import pandas as pd
def recommend_movies_by_genre(genre, movie_df, n=5):
    """
    Recommends the top n movies based on a specific genre.

    Args:
        genre (str): The genre to filter movies by.
        movie_df (pd.DataFrame): The DataFrame containing movie information (with 'genres' column).
        n (int): The number of top movies to recommend.

    Returns:
        pd.Series: A Series of movie titles recommended.
    """
    # Filter movies that contain the specified genre in their 'genres' string
    genre_movies = movie_df[movie_df['genres'].str.contains(genre, na=False)]

    # For simplicity, we'll recommend the top-rated movies within that genre.
    # This assumes we have a measure of "top-rated". If not, you might need to
    # use rating data or a different metric.
    # Assuming higher 'movieId' or simply the first ones that appear in the filtered list
    # are sufficient for a basic recommendation within the genre for this example.
    # A more sophisticated approach would involve average ratings, popularity, etc.

    # For a simple recommendation based on genre alone, we'll just take the first n
    # movies found in that genre. If you have rating data associated with the movies
    # themselves (like average ratings), you would sort by that.
    recommended_movies = genre_movies['title'].head(n)

    return recommended_movies

# Example: Recommend Top 5 Action movies
action_genre = "Action"
top_action_movies = recommend_movies_by_genre(genre=action_genre, movie_df=movie_df, n=5)
print(f"Top 5 {action_genre} Movie Recommendations:\n")
print(top_action_movies)

# Example: Recommend Top 5 Comedy movies
comedy_genre = "Comedy"
top_comedy_movies = recommend_movies_by_genre(genre=comedy_genre, movie_df=movie_df, n=5)
print(f"\nTop 5 {comedy_genre} Movie Recommendations:\n")
display(top_comedy_movies)

Top 5 Action Movie Recommendations:

5                 Heat (1995)
8         Sudden Death (1995)
9            GoldenEye (1995)
14    Cutthroat Island (1995)
19         Money Train (1995)
Name: title, dtype: object

Top 5 Comedy Movie Recommendations:



,title
0,Toy Story (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
6,Sabrina (1995)


In [16]:
# Recommend Top 5 Movies released on certain year
def recommend_movies_by_year(year, movie_df, n=5):
    """
    Recommends the top n movies released in a specific year.

    Args:
        year (int): The release year to filter movies by.
        movie_df (pd.DataFrame): The DataFrame containing movie information (with 'title' column).
        n (int): The number of top movies to recommend.

    Returns:
        pd.Series: A Series of movie titles recommended.
    """
    # Filter movies that contain the specified year in their 'title' string
    year_movies = movie_df[movie_df['title'].str.contains(f'({year})', na=False)]

    # For a simple recommendation based on year alone, we'll just take the first n
    # movies found for that year. If you have rating data associated with the movies
    # themselves (like average ratings), you would sort by that.
    recommended_movies = year_movies['title'].head(n)

    return recommended_movies

# Example: Recommend Top 5 movies released in 1995
release_year = 1995
top_year_movies = recommend_movies_by_year(year=release_year, movie_df=movie_df, n=5)
print(f"Top 5 Movie Recommendations from {release_year}:\n")
display(top_year_movies)

Top 5 Movie Recommendations from 1995:



,title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
